<a href="https://colab.research.google.com/github/pavi-ninjaac/Specrom_Analysis/blob/main/Internship_works/week3/text_classification_109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#get the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

#sklearn functions
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import LabelEncoder

#sklearn vectorization packages
from sklearn.

In [20]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/Text_classification_news_article/rishika_csv_different_computer_with_iptc.csv',index_col=0)

shape = data.shape
columns = list(data.columns)

print("the shape of the dataset is--->",shape)
print('The features are-------------->',columns)

data.head(4)


the shape of the dataset is---> (10761, 8)
The features are--------------> ['position', 'query', 'url', 'title', 'snippet', 'combined_text', 'level1_labels', 'level2_labels']


,position,query,url,title,snippet,combined_text,level1_labels,level2_labels
0,1,"arts, culture, entertainment and media",https://www.bbc.com/news/entertainment_and_arts,Entertainment & Arts - BBC News,Get the latest BBC Entertainment and Arts news...,Entertainment & Arts - BBC News Get the latest...,"arts, culture, entertainment and media",NaN
1,2,"arts, culture, entertainment and media",https://www.champaigncounty.org/list/ql/arts-c...,"Arts, Culture & Entertainment | Champaign Coun...",Champaign County Chamber of Commerce - IL 303 ...,"Arts, Culture & Entertainment | Champaign Coun...","arts, culture, entertainment and media",NaN
2,3,"arts, culture, entertainment and media",https://www.bls.gov/oes/current/oes270000.htm,"Arts, Design, Entertainment, Sports, and Media...","27-0000 Arts, Design, Entertainment, Sports, a...","Arts, Design, Entertainment, Sports, and Media...","arts, culture, entertainment and media",NaN
3,4,"arts, culture, entertainment and media",https://www.smc.edu/academics/areas-of-interes...,"Arts, Media, & Entertainment - Santa Monica Co...",Let your imagination run wild and free — explo...,"Arts, Media, & Entertainment - Santa Monica Co...","arts, culture, entertainment and media",NaN


In [10]:
print("The nunber of category----------->",data['level1_labels'].nunique())
print("The nunber of second category---->",data['level2_labels'].nunique())

The nunber of category-----------> 17
The nunber of second category----> 109


In [14]:
print("The nunber of quieries---->",data['query'].nunique())

The nunber of quieries----> 1296


In [16]:
data['position'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [22]:
needed_columns = ['combined_text','query','level1_labels','level2_labels']
data = data[needed_columns].copy()
data.head(4)

,combined_text,query,level1_labels,level2_labels
0,Entertainment & Arts - BBC News Get the latest...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
1,"Arts, Culture & Entertainment | Champaign Coun...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
2,"Arts, Design, Entertainment, Sports, and Media...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
3,"Arts, Media, & Entertainment - Santa Monica Co...","arts, culture, entertainment and media","arts, culture, entertainment and media",NaN


# Handling dublicate values

In [33]:
#delete the dublicate values
data.duplicated().sum() # count the total duplicate samples

364

In [34]:
data.drop_duplicates(keep='last',inplace=True)

In [35]:

#there can be dublicate of author names so check for the dublicate headline and short discription
data.duplicated(subset=[ 'combined_text']).sum()

759

In [36]:
data.drop_duplicates(subset='combined_text' , keep ='last' , inplace=True)

In [37]:
print("THe length of the datset after dublicate deletion------>",data.shape)

THe length of the datset after dublicate deletion------> (9638, 4)


# Handling null values

In [38]:
data.isnull().sum()

combined_text      1
query              0
level1_labels      0
level2_labels    133
dtype: int64

In [39]:
data.dropna(axis=0 ,inplace=True , how='any')

In [40]:
print("THe length of the datset after null values deletion------>",data.shape)

THe length of the datset after null values deletion------> (9504, 4)


In [42]:
data[data['combined_text'] == ' ']

,combined_text,query,level1_labels,level2_labels


# Clean the dataset

In [25]:
def preprocessor_text(text):
    if isinstance((text), (str)):
        text = re.sub('<[^>]*>', ' ', text)
        text = re.sub('[\W]+', ' ', text.lower())
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = re.sub('<[^>]*>', '', text[i])
            temp_text = re.sub('[\W]+', '', temp_text.lower())
            return_list.append(temp_text)
        return(return_list)

data['combined_text'] = data['combined_text'].apply(preprocessor_text)

In [26]:
data.head(4)

,combined_text,query,level1_labels,level2_labels
0,entertainment arts bbc news get the latest bbc...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
1,arts culture entertainment champaign county ch...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
2,arts design entertainment sports and media occ...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN
3,arts media entertainment santa monica college ...,"arts, culture, entertainment and media","arts, culture, entertainment and media",NaN


In [30]:
X = data['combined_text']
y = data['query']
#label encode it for classification
label = LabelEncoder()
y = label.fit_transform(y)

In [31]:
X_train,y_train,X_test,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [32]:
print("The shape of the X train----------->",X_train.shape)
print("The shape of the y train----------->",y_train.shape)
print("The shape of the X test------------>",X_test.shape)
print("The shape of the y test------------>",y_test.shape)

The shape of the X train-----------> (8608,)
The shape of the y train-----------> (2153,)
The shape of the X test------------> (8608,)
The shape of the y test------------> (2153,)


# tfidf Vectorization